In [1]:
import numpy as np
import pandas as pd
import math
import random
from PIL import Image
import cv2
import os
import matplotlib.pyplot as plt
import shutil
import json
from shapely.geometry import Polygon, MultiPolygon
import tensorflow as tf


class Point_Source_Generator:
  #This is a class which generates a source image based on some (latitude, longitude) coordinate describing the source
  #and number of photons detected, everything else is fixed, but can be made parameters later
  #How/the order to call the class:
  #Make class instance, call fill_counts(), call convert_to_jpg(), call get_num_pixels(), finally call gen_xml_annot()

  #How parameters should be varied:
  #counts - [50, 100, 1000, 10000]    
  #b_source - [-20, 0, 20]        
  #l_source - [60, 80, 100]       
  #psf_sigma - [0.5, 1.0, 1.5, 2.0]  
  #consts - [0.1, 0.8, 3.0, 5.0, 7.0, 10.0]
  #pixel size need to be smaller than 1
  #number of segments must be 20 or less to avoid errors!
  
  seed = 1234 #this is the seed used to seed the poisson generator
  random.seed() #this uses system time, but should try to use a seed eventually in case we need to reproduce images

  #UNCHANGING QUANTITIES:
  RAND_MAX = 2147483647 #this value is RAND_MAX in C++ (but is different per compiler I think)

  def __init__(self, folder_path, b_source, l_source, source_counts, namef, psf_sigma,
               constant_ = 20.0, num_sources=1, segments = 20, spread = 10.0, pixel_size = 0.2, l_min = 10.0, l_max = 110.0, 
               b_min = -50.0, b_max =50.0):
    #constructor
    if num_sources > 1:
      assert len(b_source) == len(l_source), "Lengths of B/L arrays are NOT equal."
      assert len(b_source) == len(source_counts), "Length of source counts array is NOT equal."

    self.b_source= b_source #if multiple sources, will be list
    self.l_source=l_source #if multiple sources, will be list
    self.source_counts = source_counts #if multiple, will be list
    self.psf_sigma = psf_sigma #this can also be  a list!

    self.l_min = l_min
    self.l_max = l_max
    self.b_min = b_min
    self.b_max = b_max
    
    self.pixel_size = pixel_size

    self.namef = namef
    self.spread = spread
    self.segments = segments
    self.folder_path = folder_path
    self.constant_ = constant_
    self.num_sources = num_sources

    self.nl = (l_max - l_min)/pixel_size
    self.nb = (b_max - b_min) / pixel_size #pixel_size +1 ?
    self.norm = 0.0
  
  def debug_type(self, anything):
    #debug function to return data type and if np array dtype
    n = "nothing"
    if anything is None:
      print("NOTHING HERE")
      return n
    else:
      print("type:" , type(anything))
      if type(anything) is np.ndarray:
        print("np array dtype: ", anything.dtype)
        print("np array dim: ", np.shape(anything))
        self.debug_np_print(anything)
    return n

  def debug_np_print(self, np_ar):
    for x in np_ar:
      for y in x:
        print(y, end =" ")
      print()

  def call_once(self):
    print("Integer Poisson counts: ")
    self.debug_np_print(self.int_counts)

  def gaussian_fill(self, i, j):
    #2D gaussian fill generator, takes coordinates and generates 2D gaussian
    i = float (i)
    j = float (j)
    l = self.l_min + i*self.pixel_size 
    b = self.b_min + j*self.pixel_size
    angle = math.sqrt(pow(l - self.l_source, 2) + pow(b - self.b_source, 2))
    temp_ = math.exp( -pow(angle,2)/(2*pow(self.psf_sigma,2))) #Gaussian calculation 
    return temp_

  def gaussian_fill_m(self, i, j, ind):
    #2D gaussian fill generator for multiple sources
    i = float (i)
    j = float (j)
    l = self.l_min + i*self.pixel_size 
    b = self.b_min + j*self.pixel_size
    angle = math.sqrt(pow(l - self.l_source[ind], 2) + pow(b - self.b_source[ind], 2))
    temp_ = math.exp( -pow(angle,2)/(2*pow(self.psf_sigma[ind],2))) #Gaussian calculation 
    return temp_

  def poisson_fill(self, x, init=1, p = 0.05 ): #p = 0.05
    #poisson random number generator, takes some input number x and generates an integer 
    m = int (x/p) #double input divided by probability of an event (0.05 in this case)
    if m<1: 
      m =1  #need at least one trial, m will give the number of trials
    p = x / (float (m)) #updating the probability from the decided number of trials
    if init == 1:
      init=0
    P = int (p * self.RAND_MAX) #P = probability times highest randomly generated int
    iter = 0
    N = 0
    for i in range(m):
      iter += 1
      temp = random.randint(0, self.RAND_MAX)
      if (temp < P):
        N +=1
    return N

  def make_source(self, index):
    norm = 0.0
    counts = np.zeros((round(self.nb), round(self.nl)), dtype = float)
    for (x,y), value in np.ndenumerate(counts):
      tmp_ = self.gaussian_fill_m(x, y, index)
      counts[x,y] = tmp_
      norm += tmp_
    counts = np.multiply(counts, (self.source_counts[index]/norm))
    return counts

  def fill_counts(self):
    #fills the counts np array with floats and int_counts with ints, sets norm
    list_counts = []
    self.counts = np.zeros((round(self.nb), round(self.nl)), dtype = float)
    
    if self.num_sources > 1:
      for i in range(self.num_sources):
        new_counts = self.make_source(i)
        list_counts.append(new_counts)
      for i in list_counts:
        for (x, y), value in np.ndenumerate(self.counts):
          self.counts[x, y] += i[x, y]
      for (x,y), value in np.ndenumerate(self.counts):
        self.counts[x, y] += self.constant_     
    
    else:
      for (x,y), value in np.ndenumerate(self.counts):
        tmp_ = self.gaussian_fill(x, y)
        self.counts[x,y] = tmp_
        self.norm += tmp_ 
      for (x,y), value in np.ndenumerate(self.counts):
        self.counts[x, y] += self.constant_
      counts = np.multiply(counts, (self.source_counts/self.norm))

    self.int_counts = np.zeros((round(self.nb), round(self.nl)), dtype=int)

    for (x,y), value in np.ndenumerate(self.int_counts):
      tmp = self.poisson_fill(self.counts[x, y])
      self.int_counts[x , y] = tmp


  def convert_to_jpg(self):
    #takes np array and then converts to jpg image and returns name of file saved
    #For some reason, saving the images in RGB format produces very different looking images
    #Why does this happen?
    #we may have to normalize pixel values from 0-1 instead of 0-255
    nn = self.int_counts.astype(float) * 255.0 / float (self.int_counts.max())
    nn = np.rint(nn)
    nn = nn.astype(np.uint8)
    im = Image.fromarray(nn)
    jpeg_nme = self.namef + ".jpeg"
    jpeg_path = self.folder_path + jpeg_nme
    im.save(jpeg_path,"JPEG")
    print("File with name {}.jpeg generated.".format(self.namef))


  def get_num_pixels(self):
    #gets the total number of pixels + the pixels along the sides of the image
    #alternatively we could use nl, nb variables for width/height but if these ever get mixed up in the class
    #this could be dangerous, with Image.open() we are more likely to catch mistakes here

    filepath = self.folder_path+ self.namef + ".jpeg"

    width, height = Image.open(filepath).size
    self.num_pixels = width*height
    self.num_pix_w = width
    self.num_pix_h = height

    if width < 300 or height < 300:
      print("Image size has too few pixels - adjust model for {} x {} image".format(width, height))
    if width > 2000 or height > 2000:
      print("Image size has too many pixels - adjust model for {} x {} image".format(width, height))
    print("Total pixels: {}, height pixels: {}, width pixels: {}".format(self.num_pixels, self.num_pix_h, self.num_pix_w))
    return height, width


  def get_xyminmax(self):
    #gets the xmin, xmax, ymin, and ymax pixels of the source in the image
    self.xmin = []
    self.ymin = []
    self.xmax = []
    self.ymax = []
    self.wid_ = []
    self.heig_ = []
    self.xcsource = []
    self.ycsource = []
    self.temp_spr = []

    for i in range(self.num_sources):
      self.temp_spr.append(self.spread*self.psf_sigma[i])
      x = abs(self.b_min - self.b_source[i])
      y = abs(self.l_min - self.l_source[i])
      xcs = float (x) / self.pixel_size
      ycs = float (y) / self.pixel_size
      ymax_ = ycs + self.temp_spr[i]
      xmax_ = xcs + self.temp_spr[i]
      xmin_ = xcs - self.temp_spr[i]
      ymin_ = ycs - self.temp_spr[i]
      self.xcsource.append(xcs)
      self.ycsource.append(ycs)
      self.xmin.append(xmin_)
      self.ymin.append(ymin_)
      self.xmax.append(xmax_)
      self.ymax.append(ymax_)
      self.wid_.append(abs(xmax_ - xmin_))
      self.heig_.append(abs(ymax_ - ymin_))
      print("x coord of source #{} is {} and y coord is {}".format(i+1, self.xcsource[i], self.ycsource[i]))
    bbox = [self.xmin, self.ymin, self.wid_, self.heig_]
    return bbox


  def mask_circle_pos(self, x, radius, i):
    sqrt_ = pow(radius, 2) - pow((x - self.xcsource[i]), 2)
    y = math.sqrt(sqrt_) + self.ycsource[i]
    return y 


  def mask_circle_neg(self, x, radius, i):
    sqrt_ = pow(radius, 2) - pow((x - self.xcsource[i]), 2)
    y = self.ycsource[i] - math.sqrt(sqrt_)
    return y 


  def return_JSON_entry(self):
    filename = self.namef + ".jpeg"
    area = []
    self.segment = []
    for i in range(self.num_sources):
      self.segment.append([])
    for i in range(self.num_sources):
      radius = self.temp_spr[i]
      area.append(pow(radius,2) * 2.0 * math.pi)
      seg_leg = (self.xmax[i] - self.xmin[i]) / float (self.segments)
      xstart = self.xmin[i]
      ystart = self.ycsource[i]
      self.segment[i].append(xstart)
      self.segment[i].append(ystart)
      y_ind = 1

      for s in range(self.segments):
        xstart += seg_leg
        self.segment[i].append(xstart)
        if xstart > self.xmax[i]:
          y = self.ycsource[i]
        else:
          y = self.mask_circle_pos(xstart, radius, i)
        self.segment[i].append(y)
        y_ind += 2

      xstart = self.xmax[i] 
      ystart = self.ycsource[i]
      print("NEGATIVE NOW - starting point is now ({}, {}), radius is still {}, segleg is still {}".format(xstart, ystart, radius, seg_leg))
      for s in range(self.segments):
        xstart -= seg_leg
        self.segment[i].append(xstart)
        if xstart < self.xmin[i]:
          y = self.ycsource[i]
        else:
          y = self.mask_circle_neg(xstart, radius, i)
        self.segment[i].append(y)
        y_ind += 2

    return filename, self.segment, area

In [2]:
#Function to visualize images in a folder
def visualize_imgs(path_to_folder):
  items = os.listdir(path_to_folder)
  for each_image in items:
    if each_image.endswith(".jpeg"):
      full_path = path_to_folder + each_image
      image = cv2.imread(full_path)
      image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
      plt.figure()
      plt.imshow(image)
      plt.title(each_image)

def visualize_img(path_to_img):
  image = cv2.imread(path_to_img)
  image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
  plt.figure()
  plt.imshow(image)

In [3]:
B_ = [  -5, -3, -1, 0, 1, 2, 3, 5 ] #B goes from -50 to 50
L_ = [50, 51, 53, 55, 56, 57, 58, 60] #L goes from 10 to 110
psf = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0]
constant = [2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0]
num_src = [3, 4, 5, 6, 7, 8, 9]
counts = [50, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

#len(psf) = ?
#len(constant) = 10
#len(num_src) = 8
#len(counts) = 20
#len(B_) = ?
#len(L_) = ?

psf_1 = psf[:13] #0.5 - 1.5
constant_1 = constant[2:] #4.0 - 15.0
num_src_1 = num_src[2:] #5 - 9
counts_1 = counts[5:15] #500 - 5000
print("counts:", counts_1)
print("psf:", psf_1)
print("constant:", constant_1)
print("num of sources:", num_src_1)

counts: [500, 600, 700, 800, 900, 1000, 2000, 3000, 4000, 5000]
psf: [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 1.1, 1.2, 1.3, 1.4, 1.5]
constant: [4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0]
num of sources: [5, 6, 7, 8, 9]


In [4]:
!mkdir train

rm: cannot remove '/content/train/': No such file or directory


In [6]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [10]:
BB = -10
LL = 50
CC = 1000
name_new = "imtest"
PSF = 1.8
cnst = 10
ns = 2

temp = Point_Source_Generator("/content/train/", BB, LL, CC, name_new, PSF, cnst, ns)
temp.fill_counts()
temp.convert_to_jpg()

UnboundLocalError: ignored

In [8]:
data = {}
data['info'] = []
data['images'] = []
data['categories'] = []
data['annotations'] = []

json_path = "TRAIN4.json"
f = open(json_path, "w")

data['info'].append({
    'description': "This is a segmented train dataset of simulated gamma ray point sources.",
    'url' : "https://sites.google.com/view/dianabhorangic/home",
    'version' : 4.0,
    'year': 2021,
    'contributor': "Diana Horangic",
    'date_created': 2021
})
data['categories'].append({
    "supercategory": 'point source',
    "id": 1,
    "name": 'point source'
})

name_ = "img"
mit = -1
ids = 1

for i in range(4): #200 images
  for ns in num_src_1:
    mit += 1
    name_new = name_ + str (mit)
    BB = []
    LL = []
    CC = []
    PSF = []
    bind = random.sample(range(0, len(B_)), ns)
    lind = random.sample(range(0, len(L_)), ns)
    cind = [random.randint(0, len(counts_1)-1)  for i in range(ns)]
    psfind = [random.randint(0, len(psf_1)-1)  for i in range(ns)]
    for i in bind:
      BB.append(B_[i])
    for j in lind:
      LL.append(L_[j])
    for k in psfind:
      PSF.append(psf_1[k])
    for m in cind:
      CC.append(counts_1[m])
    cnst = constant_1[random.randint(0, len(constant_1)-1)]
    print("cnst is ", cnst)

    temp = Point_Source_Generator("/content/train/", BB, LL, CC, name_new, PSF, cnst, ns)
    temp.fill_counts()
    temp.convert_to_jpg()
    height, width = temp.get_num_pixels()
    bbox = temp.get_xyminmax()
    filename, segment, area = temp.return_JSON_entry()

    data['images'].append({
            'height': height,
            'width': width,
            'id': mit,
            'file_name': filename
    })

    for i in range(ns):
      bbox_ = [bbox[0][i], bbox[1][i], bbox[2][i], bbox[3][i]]
      data['annotations'].append({
              'segmentation': [segment[i]],
              'area': area[i],
              'image_id': mit, #should match id for image
              'bbox': bbox_,
              'bbox_mode': 1,
              'category_id': 1, #1 is point source
              'id': ids #increase this for each segmentation
      })
      ids += 1
      print("segment id is {}".format(ids))
      print("bounding box:", bbox_)
      print("length of segment list is:", len(segment[i]))
  

cnst is  7.0
File with name img0.jpeg generated.
Total pixels: 250000, height pixels: 500, width pixels: 500
x coord of source #1 is 225.0 and y coord is 230.0
x coord of source #2 is 245.0 and y coord is 240.0
x coord of source #3 is 265.0 and y coord is 205.0
x coord of source #4 is 260.0 and y coord is 235.0
x coord of source #5 is 275.0 and y coord is 225.0
NEGATIVE NOW - starting point is now (240.0, 230.0), radius is still 15.0, segleg is still 1.5
NEGATIVE NOW - starting point is now (249.0, 240.0), radius is still 4.0, segleg is still 0.4
NEGATIVE NOW - starting point is now (269.0, 205.0), radius is still 4.0, segleg is still 0.4
NEGATIVE NOW - starting point is now (265.0, 235.0), radius is still 5.0, segleg is still 0.5
NEGATIVE NOW - starting point is now (287.0, 225.0), radius is still 12.0, segleg is still 1.2
segment id is 2
bounding box: [210.0, 215.0, 30.0, 30.0]
length of segment list is: 82
segment id is 3
bounding box: [241.0, 236.0, 8.0, 8.0]
length of segment list

ValueError: ignored

In [ ]:
json.dump(data, f, indent=4)
f.close() 

In [ ]:
!zip -r /content/TRAIN4.zip /content/train
!cp "/content/TRAIN4.zip" "/content/drive/MyDrive/DOESULI"
!cp "/content/TRAIN4.json" "/content/drive/MyDrive/DOESULI"